In [1]:
import numpy as np
import pandas as pd
import re
import json
from csv import reader

In [2]:
network = {'nodes':[],'links':[]}
nodes = {}
node_self = {}
threshold = 9
ifh = open('boarddesignerOtherwayBigger10.csv','r')
first = True
filereader = reader(ifh, delimiter=',', quotechar='"')
for row in filereader:
    if first:
        first = False
        brands = row[1:]
        continue
    assert len(row) == len(brands) + 1
    source = row[0]
    for brand_idx in range(len(row)-1):
        if source != brands[brand_idx]:
            if float(row[brand_idx+1]) > threshold:
                if source not in nodes:
                    position = len(nodes)
                    nodes[source] = position
                    network['nodes'].append({'name':source})
                if brands[brand_idx] not in nodes:
                    position = len(nodes)
                    nodes[brands[brand_idx]] = position
                    network['nodes'].append({'name':brands[brand_idx]})
                link = {'source':nodes[source],
                        'target':nodes[brands[brand_idx]],
                        'value':float(row[brand_idx+1])}
#                 print source + ", " + brands[brand_idx] + ": " + row[brand_idx+1]
                network['links'].append(link)
        else:
            node_self[source] = float(row[brand_idx+1])

In [3]:
for link in network['links']:
    source = network['nodes'][link['source']]['name']
    target = network['nodes'][link['target']]['name']
#     print source
    source_value = node_self[source]
    target_value = node_self[target]
    link['jaccard'] = link['value']/(source_value + target_value - link['value'])

In [ ]:
with open('brand_network.json','w') as ofh:
    json.dump(network, ofh)

In [ ]:
brand_nodes = pd.DataFrame(network['nodes'])

In [10]:
brand_links = pd.DataFrame(network['links'])

In [ ]:
min(brand_links['jaccard'])

In [ ]:
max(brand_links['jaccard'])

In [29]:
brand_edges = brand_links[brand_links['source']<brand_links['target']]

In [38]:
import igraph

In [63]:
brandG = igraph.Graph(brand_edges[['source','target']].values.tolist())

In [64]:
brandG.es['weight'] = brand_edges[['jaccard']].values.flatten().tolist()

In [65]:
brandG = brandG.as_undirected()

In [75]:
brandG_dendogram = brandG.community_fastgreedy(weights='weight')

In [76]:
brandG_clusters = brandG_dendogram.as_clustering(3)

In [77]:
brandG_clusters.q

0.113630423556076

In [79]:
subG1 = brandG_clusters.subgraph(0)

In [82]:
subG1.get_nodes()

AttributeError: 'Graph' object has no attribute 'get_nodes'